In [1]:
import json
import numpy as np
import pandas as pd
import keras
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, Dropout, Input, Flatten, Embedding, LSTM
from keras.utils import np_utils
from keras.utils import to_categorical
from sklearn import preprocessing
import sklearn
import time
from random import shuffle
import pickle

C:\Users\kevin\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
os.getcwd()

'E:\\GT\\JrDesign\\demo\\grocery_e-receipt\\approaches\\LSTM'

In [3]:
with open("../../synthetic_data/data/final_train_labels.json") as f:
    data = json.load(f)

In [4]:
data = data[:300000]
len(data)

300000

In [5]:
x = [ea[1] for ea in data]
y = [ea[0] for ea in data]
df = pd.DataFrame(data={"X": x, "Y": y})
print (len(df))
df.head()

300000


,X,Y
0,LGHT NNFT YGRT,LIGHT NONFAT YOGURT
1,RSN LMND CRNCH T CLSTRS,RAISIN ALMOND CRUNCH OAT CLUSTERS
2,BNS & GCML,BEANS & GUACAMOLE
3,WHT MT CRNBRRY PCN CHCKN SLD,WHITE MEAT CRANBERRY PECAN CHICKEN SALAD
4,FRMHS-STYL SSG SCRMBL,FARMHOUSE-STYLE SAUSAGE SCRAMBLE


In [6]:
# Y Vocab
labels = y
le = preprocessing.LabelEncoder()
df['encoded_y'] = le.fit_transform(y)
print ("# categories = ", max(df.encoded_y.tolist())+1)
df.head(5)

# categories =  105730


,X,Y,encoded_y
0,LGHT NNFT YGRT,LIGHT NONFAT YOGURT,54513
1,RSN LMND CRNCH T CLSTRS,RAISIN ALMOND CRUNCH OAT CLUSTERS,74612
2,BNS & GCML,BEANS & GUACAMOLE,7322
3,WHT MT CRNBRRY PCN CHCKN SLD,WHITE MEAT CRANBERRY PECAN CHICKEN SALAD,102494
4,FRMHS-STYL SSG SCRMBL,FARMHOUSE-STYLE SAUSAGE SCRAMBLE,33857


In [7]:
# X Vocab
MAX_SEQ_LEN = 25
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 256

flattened = np.asarray(x).reshape(-1)
t = Tokenizer(num_words=MAX_NUM_WORDS)
t.fit_on_texts(flattened)

## see encoded X --> do batch encoding, cannot load into memory
encoded_x = t.texts_to_sequences(flattened)
train_x = pad_sequences(encoded_x, maxlen=MAX_SEQ_LEN, dtype='int16')
train_x

array([[   0,    0,    0, ...,  298,  804,  144],
       [   0,    0,    0, ...,  400,   24,  949],
       [   0,    0,    0, ...,    0,   31, 1434],
       ...,
       [   0,    0,    0, ...,   58,    3,   28],
       [   0,    0,    0, ...,  466, 2291,    2],
       [   0,    0,    0, ...,    8,   49,   10]], dtype=int16)

In [8]:
train_y = to_categorical(df.encoded_y.tolist(), num_classes=max(df.encoded_y.tolist())+1, dtype="int8")
train_y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [9]:
model = Sequential()
model.add(Embedding(MAX_NUM_WORDS, EMBEDDING_DIM, input_length=MAX_SEQ_LEN))
model.add(LSTM(100))
model.add(Dense(len(train_y[0]), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 256)           5120000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               142800    
_________________________________________________________________
dense_1 (Dense)              (None, 105730)            10678730  
Total params: 15,941,530
Trainable params: 15,941,530
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
print (train_x.shape)
print (train_y.shape)

(300000, 25)
(300000, 105730)


In [11]:
start = time.time()
model.fit(train_x, train_y, epochs=1)
model.save("models/lstm_300k_epochs_1.h5")
print ("Took: ", time.time()-start)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
300000/300000 [==============================] - 2730s 9ms/step - loss: 11.8086 - acc: 1.0000e-05
Took:  2732.5319199562073


# Evaluation

In [12]:
model = load_model("models/lstm_300k_epochs_1.h5")
test_num = 300
preds = model.predict(train_x[:test_num])
pred_labels = [[np.argmax(x)] for x in preds]

In [13]:
pred_titles = le.inverse_transform(pred_labels)
pred_titles

C:\Users\kevin\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([['STRAINED NONFAT YOGURT'],
       ['Honeysuckle White Turkey Breast Tenderloins'],
       ['CHOCOLATE CHEEZECAKE'],
       ['CLASSIC CHEESE DUO HAND CRAFTED PIZZA'],
       ['BLUE AGAVE NECTAR SWEETENER'],
       ['GARLIC'],
       ['CREAM CHEESE MUFFINS'],
       ['BAKED FISH SNACK'],
       ['GREEK ICED TEA'],
       ['MIXED BERRY GRANOLA BARS'],
       ['FINE GRAIN HIMALAYAN PINK SALT'],
       ['MORSELS WHITE BAKING CHIPS'],
       ['TEDDY BEAR GRAHAMS'],
       ['PECAN NUT TOPPING'],
       ['ONION & CHIVES BLENDED IN CREAMY CHEDDAR'],
       ['ALOE SMOOTHIE MADE WITH YOGURT'],
       ['EGGNOG PRALINE CHEESE CAKE'],
       ['NONFAT GREEK YOGURT WITH STRAWBERRIES & GRANOLA'],
       ['SOUTHERN CLASSICS CUT OKRA'],
       ['CALIFORNIA PITTED MEDIUM RIPE OLIVES'],
       ['FUDGE RIPPLE ICE CREAM'],
       ['BAVARIAN PRETZELS'],
       ['SPAGHETTI NESTS WITH SPRING WATER'],
       ['LENTILS HERITAGE BEANS'],
       ['Gold Peak Sweetened Black Tea Bottle'],
       ['SWEET SANDE

In [15]:
analysis_df = pd.DataFrame({'pred_x': pred_titles.tolist(), 'x': x[:test_num], 'y': y[:test_num]})
analysis_df = analysis_df[['x', 'y', 'pred_x']]

In [16]:
analysis_df

,x,y,pred_x
0,LGHT NNFT YGRT,LIGHT NONFAT YOGURT,[STRAINED NONFAT YOGURT]
1,RSN LMND CRNCH T CLSTRS,RAISIN ALMOND CRUNCH OAT CLUSTERS,[Honeysuckle White Turkey Breast Tenderloins]
2,BNS & GCML,BEANS & GUACAMOLE,[CHOCOLATE CHEEZECAKE]
3,WHT MT CRNBRRY PCN CHCKN SLD,WHITE MEAT CRANBERRY PECAN CHICKEN SALAD,[CLASSIC CHEESE DUO HAND CRAFTED PIZZA]
4,FRMHS-STYL SSG SCRMBL,FARMHOUSE-STYLE SAUSAGE SCRAMBLE,[BLUE AGAVE NECTAR SWEETENER]
5,PPLS,APPLES,[GARLIC]
6,TQNS CHS STCKS,TEQUENOS CHEESE STICKS,[CREAM CHEESE MUFFINS]
7,BNLSS PRK TNDRLN,BONELESS PORK TENDERLOIN,[BAKED FISH SNACK]
8,DCF CD T,DECAF ICED TEA,[GREEK ICED TEA]
9,MXD BRRY GRNL BRS,MIXED BERRY GRANOLA BARS,[MIXED BERRY GRANOLA BARS]


In [ ]:
np.save('pickled/labelencoder_classes.npy', le.classes_)

In [ ]:
with open('pickled/tokenizer.pickle', 'wb') as handle:
    pickle.dump(t, handle, protocol=pickle.HIGHEST_PROTOCOL)